# Listado usuarios activos
Este reporte extrae un archivo en formato .csv con los datos de todos los usuarios activos.

Definición de "Usuario Activo": Usuario que ha interactuado al menos una vez con el curso.

In [ ]:
import pandas as pd

def get_db():
    '''Access MongoDB'''
    from pymongo import MongoClient
    client = MongoClient('mongodb://XXX.XX.X.X:XXXXX')
    db = client.edx
    return db

def get_courses(db):
    '''Get the collections that refers to courses'''
    collections = db.collection_names(include_system_collections=True)
    collections =  list(collections) 
    #print "Collections: \n", collections
    for item in collections: #Remove those that are not courses.
        if item[0:3] != "idb":
            #print item
            collections.remove(item)

    #collections.remove("reports") #For some reason it does not remove reports collection.
    courses = []
    for i in collections:
        c = i.split(".")
        courses.append(c[0])

    courses = set(courses)
    print "Courses: \n", courses
    return courses, collections

In [ ]:
def get_active_users(db, collections):
    '''Get only the users that have use at least onece the platform'''
    #Estadisticas = pd.DataFrame(columns=['Curso','Número de usuarios activos','Total de usuarios registrados','Porcentaje usuarios activos']
    
    df_all = pd.DataFrame()
    
    for index, item in enumerate(courses): #Loop for each course
        collection_courseware = item + ".courseware"
        if collection_courseware in collections:
                
            print item
            #Get name of the course from course_names
                                            
            #Get information from courseware
            courseware = db[item].courseware.find({}, {"id":1, "student_id":1}) #Save data in a variable
            df =  pd.DataFrame(list(courseware)) #Transform it into dataframe
            df["course_id"] = item #Add a column with course_id

            user_count = pd.DataFrame(df['student_id'].value_counts()) #Count number of times each user interacts with the platform
            # Prepare variable
            user_count["student_id2"] = user_count.index # Name indexes as a new columnd
            user_count.columns = ["Repetitions", "student_id"] #Name columns

            df2 = pd.merge(df, user_count, on = "student_id", how = "left") #Join original dataframe with counts
            df2 = df2.sort_values(by = "student_id") # Sort it just to check it.
            df2 = df2.drop_duplicates(subset=['student_id'], keep='first') #Drop duplicates values.


            # Get personal information from collection.users
            users = db[item].users.find({"is_staff":0}, {"id":1, "username":1, "first_name":1, "last_name":1, "email":1})
            users_df = pd.DataFrame(list(users))

            df3 = pd.merge(df2, users_df, left_on = "student_id", right_on = "id", how = "left")
            df3 = df3.drop('_id_x', 1)
            df3.rename(columns = {'id_x':'_id'}, inplace = True)
            df3 = df3.drop('_id_y', 1)
            df3 = df3.drop('id_y', 1)
            #print "df3:\n",df3.head(20)
            
            # Get certificates information from collection.certificates
            certificates = db[item].certificates.find({}, {"id":1, "user_id":1, "status":1})
            certificates_df = pd.DataFrame(list(certificates))
            #print certificates_df
            
            if len(certificates_df) != 0:
                print('DataFrame is not empty!')
                df4 = pd.merge(df3, certificates_df, left_on = "student_id", right_on = "user_id", how = "left")
            #df3 = df3.drop('_id_x', 1)
            #df3.rename(columns = {'id_x':'_id'}, inplace = True)
            #df3 = df3.drop('_id_y', 1)
            #df3 = df3.drop('id_y', 1)
            
                frames = [df_all, df4]
                df_all = pd.concat(frames)
                #print df_all
            
    Name_csv = "All_Active_users.csv"
    df_all.to_csv(Name_csv, sep='\t', encoding='utf-8')
            
        

In [ ]:
if __name__=="__main__":
    db = get_db() #Access the db
    courses, collections = get_courses(db) #Access the collections and bring only the courses collections
    get_active_users(db, collections)
    print "HECHO"